# Setup
This code is inspired by https://www.kaggle.com/CVxTz/cnn-starter-nasnet-mobile-0-9709-lb

In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='grade3',context='paper', fscale=1, spines=True, gridlines='-',ticks=True, grid=True, figsize=(6, 6)) #somehow calling this once is not sufficient...
plotcolor = (0, 0.6, 1.0)

import sys
sys.path.append('../') 

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%reload_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

from tqdm import tqdm_notebook
from keras_tqdm import TQDMNotebookCallback
from imgaug import augmenters as iaa
import imgaug as ia
import numpy as np 
import pandas as pd 
import os,cv2,keras
from glob import glob
from random import shuffle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import backend as K
from Models.NASNet_mobile import *
from Analysis.analyze_test_results import *
from Utils.CLR.clr_callback import *

In [ ]:
#config
np.random.seed(42)
model_path = "../Out/nasnet.h5"
batch_size = 150

#Utility functions
def chunker(seq, size): #this is useful for iterating
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))
def get_id_from_file_path(file_path): #returns id for a given path
    return file_path.split(os.path.sep)[-1].replace('.tif', '')
def auc(y_true, y_pred): # AUC metric for keras, might be a little slow though (it approximates it during training too)
    auc = tf.metrics.auc(y_true, y_pred,summation_method='careful_interpolation',num_thresholds=100)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

# Load the filenames etc. for streaming training

In [ ]:
df_train = pd.read_csv("../Datasets/train_labels.csv")
df_test = pd.read_csv("../Data/compare_with_github/test_labels_from_github.csv") #load extracted test labels
id_label_map = {k:v for k,v in zip(df_train.id.values, df_train.label.values)}
id_test_map = {k:v for k,v in zip(df_test.id.values, df_test.label.values)}
labeled_files = glob('../Datasets/train/*.tif')
test_files = glob('../Datasets/test/*.tif')
train = labeled_files
val = test_files #we just validate on the extracted test data

# Setup the streaming data augmentation 

In [ ]:
#Data augmentation
def get_seq():
    sometimes = lambda aug: iaa.Sometimes(0.6, aug)
    seq = iaa.Sequential(
        [
            iaa.Fliplr(0.5), # horizontally flip 
            iaa.Flipud(0.5), # vertically flip 
            sometimes(iaa.Affine(
                scale={"x": (0.95, 1.05), "y": (0.95, 1.05)}, # scale images
                translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # translate images
                rotate=(-90, 90), # rotate 
                shear=(-5, 5), # shear by -16 to +16 degrees
            )),
            iaa.SomeOf((0, 8),
                [
                    sometimes(iaa.Superpixels(p_replace=(0, 0.5), n_segments=(20, 100))), # convert images into their superpixel representation
                    iaa.OneOf([
                        iaa.GaussianBlur((0, 0.75)), # blur images with a sigma between 0 and 3.0
                        iaa.AverageBlur(k=(3, 5)), # blur image using local means with kernel sizes between 2 and 7
                        iaa.MedianBlur(k=(3, 5)), # blur image using local medians with kernel sizes between 2 and 7
                    ]),
                    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.9, 1.1)), # sharpen images
                    iaa.Emboss(alpha=(0, 1.0), strength=(0, 0.5)), # emboss images
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.01*255), per_channel=0.5), # add gaussian noise to images
                    iaa.OneOf([
                        iaa.Dropout((0.01, 0.25), per_channel=0.5), # randomly remove up to 10% of the pixels
                        iaa.CoarseDropout((0.01, 0.03), size_percent=(0.01, 0.05), per_channel=0.2),
                    ]),
                    iaa.Add((-2, 2), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                    iaa.Multiply((0.9, 1.1), per_channel=0.5),
                ],
                random_order=True
            )
        ],
        random_order=True
    )
    return seq

#Data generator that continuously create new augmented images
def data_gen(list_files, id_label_map, batch_size, augment=False):
    seq = get_seq()
    while True:
        shuffle(list_files)
        for batch in chunker(list_files, batch_size):
            X = [cv2.imread(x) for x in batch]
            Y = [id_label_map[get_id_from_file_path(x)] for x in batch]
            if augment:
                X = seq.augment_images(X)
            X = [keras.applications.nasnet.preprocess_input(x) for x in X]
            yield np.array(X), np.array(Y)   

# Setup the model and training

In [ ]:
model = NASNet_mobile()

In [ ]:
#Setup keras training
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(0.00075),
              metrics=['accuracy',auc])

In [ ]:
# model.load_weights(model_path) #if you want to load the last model

checkpoint = keras.callbacks.ModelCheckpoint(model_path, monitor='val_auc', verbose=1, save_best_only=True, mode='max')
clr = CyclicLR(base_lr=0.00001, max_lr=0.001,step_size=1000, mode='triangular2') 

history = model.fit_generator(
    data_gen(train, id_label_map, batch_size, augment=True),
    validation_data=data_gen(val, id_test_map, batch_size),
    epochs=8, verbose=0,
    callbacks=[checkpoint,clr,TQDMNotebookCallback(leave_inner = True, leave_outer = True)],
    steps_per_epoch=len(train) // batch_size,
    validation_steps=len(val) // batch_size)

# Compute the test predictions and store submission

In [ ]:
model.load_weights(model_path) # load the best model from training

#compute test predictions
preds = []
ids = []
TTA = True #should we use TTA
for batch in tqdm_notebook(chunker(test_files, batch_size),total=len(test_files)/batch_size):
    X = [keras.applications.nasnet.preprocess_input(cv2.imread(x)) for x in batch]
    ids_batch = [get_id_from_file_path(x) for x in batch]
    X = np.array(X)
    if TTA: # 4x TTA
        preds_batch = ((model.predict(X).ravel()*model.predict(X[:, ::-1, :, :]).ravel()*model.predict(X[:, ::-1, ::-1, :]).ravel()*model.predict(X[:, :, ::-1, :]).ravel())**0.25).tolist()
    else: # No TTA
        preds_batch = model.predict(X).ravel().tolist()
    preds += preds_batch
    ids += ids_batch

In [ ]:
jtplot.style(theme='grade3',context='paper', fscale=1, spines=True, gridlines='-',ticks=True, grid=True, figsize=(6, 6)) #set plotting style again... (some bug)
analyze_test_results(ids,preds,save=True)

In [ ]:
#store the submission
df = pd.DataFrame({'id':ids, 'label':preds})
df.to_csv("../Out/submission_" + str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+".csv", index = False, header = True) #create the submission file

In [ ]:
#shutdown the pc
import subprocess
cmdCommand = "shutdown -s"
process = subprocess.Popen(cmdCommand.split(), stdout=subprocess.PIPE)

In [ ]:
# save model weights manually if desired
model.save_weights(model_path)